# Classify

Classification is a methodology that tries to match a text to the correct label. 

## Prompt based classification 

Prompt based classification is a methodology that relies purely on prompting the LLM in a specific way. 

## When should you use prompt based classification 

Some situations when you would use this methodology is when:
- The labels are easily understood (they don't require explanation or examples)
    
    An example is sentiment analysis
- The labels are not recognized by their semantic meaning
    
    E.g. Reasoning tasks like classifying contradictions
- You don't have many examples

## Example snippet 
Running the following code will instantiate a prompt based classifier, with a debug level for the log. 
Then it will classify the text given in "ClassifyInput".
The contents of the debuglog will be shown below.
The debuglog gives an overview of the steps taken to get the result.

In [15]:
from os import getenv
from aleph_alpha_client import Client
from intelligence_layer.classify import SingleLabelClassify, ClassifyInput
from pprint import pprint

client = Client(getenv("AA_API_TOKEN"))
task = SingleLabelClassify(client, "debug")
input = ClassifyInput(
    text="Beneath a canopy of stars, aliens of all kinds danced and laughed, celebrating unity in the cosmic night. \
    The asteroid throbbed with their energy, an unforgettable space party at the heart of the galaxy.", 
    labels=["Space party", "Space exploration"]
    )

output = task.run(input)
pprint(output.debug_log.model_dump())
pprint(output.scores)

{'log': [{'level': 'info',
          'message': 'Tokenized Labels',
          'value': {'Space exploration': [{'token': 'ĠSpace',
                                           'token_id': 20928},
                                          {'token': 'Ġexploration',
                                           'token_id': 42347},
                                          {'token': '<|endoftext|>',
                                           'token_id': 0}],
                    'Space party': [{'token': 'ĠSpace', 'token_id': 20928},
                                    {'token': 'Ġparty', 'token_id': 8735},
                                    {'token': '<|endoftext|>',
                                     'token_id': 0}]}},
         {'level': 'info',
          'message': 'Completion',
          'value': {'model': 'luminous-base-control',
                    'template': '### Instruction:\n'
                                'Identify a class that describes the text '
                                

## How does this implemetation work
For prompt based classification, we prompt the model multiple times with the text we want to classify and each of our classes. 
Instead of letting the model generate the class it thinks fits the text best, we ask it for the probability for each class.

To further explain this, lets start with a more familiar case.
The intuitive way to ask an LLM if it could label a text could be something like this: 

``` 
### Instruction:
Identify a class that describes the text adequately.
Reply with only the class label.

### Input:
Beneath a canopy of stars, aliens of all kinds danced and laughed, celebrating unity in the cosmic night. The asteroid throbbed with their energy, an unforgettable space party at the heart of the galaxy.

### Response:
```

The model would then answer our question, and give us a class that it thinks fits the text. 

In the case of classification, however, we already have the classes beforehand.
Because of this, all we are interested in is the probability the model would have guessed our specific classes.
To get this probability, we can prompt the model with each of our classes and ask the model to return the logprobs for the text. 

In the case of sentiment analysis a prompt could look something like this: 

``` 
### Instruction:
Identify a class that describes the text adequately.
Reply with only the class label.

### Input:
Beneath a canopy of stars, aliens of all kinds danced and laughed, celebrating unity in the cosmic night. The asteroid throbbed with their energy, an unforgettable space party at the heart of the galaxy.

<!-- The evaluated class is already filled into the prompt -->
### Response: Space party 
```

As you can see, we have pre-emptively filled in the class in our prompt.

Our request will now not generate any tokens, but instead will just return us the logprobs that the class would be generated, given the previous tokens.

In [ ]:
# This can be found in classify.py | function: _complete 
request = CompletionRequest(
    prompt=prompt_template.to_prompt(**kwargs),
    maximum_tokens=0,
    log_probs=0,
    tokens=True,
    echo=True,
)

In the case of the classes "Space exploration" and "Space party", the logprobs per label might look something like the code snippet below. 
If the code snippet above is run, this can be found in the debug log.

In [ ]:
'Space exploration': [
    {'prob': -2.7400868,
        'token': {'token': 'ĠSpace', 'token_id': 20928}},
    {'prob': -12.503683,
        'token': {'token': 'Ġexploration', 'token_id': 42347}},
    {'prob': -3.0255146,
        'token': {'token': '<|endoftext|>', 'token_id': 0}}
],
'Space party': [
    {'prob': -2.7400868,
        'token': {'token': 'ĠSpace','token_id': 20928}},
    {'prob': -0.0036826192,
        'token': {'token': 'Ġparty', 'token_id': 8735}},
    {'prob': -1.3463488,
        'token': {'token': '<|endoftext|>', 'token_id': 0}}
]

Now that we have the logprobs, we just need to do some calculations to turn them into our end score. 

To turn the logprobs into our end scores, first we normalize our probabilities. This will result in the following data structure:

In [ ]:
# Visualization here

Finally, we take the product of all the paths to get the following results:

{
    'Space exploration': 9.516251694168755e-06, 
    'Space party': 0.9999904837483058
}